In [23]:
import os

In [24]:
%pwd

'/Users/aayushaggarwal/Desktop'

In [25]:
os.chdir('/Users/aayushaggarwal/Desktop/Chicken-disease-project')

In [26]:
%pwd

'/Users/aayushaggarwal/Desktop/Chicken-disease-project'

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorbaord_log_root_dir: Path
    checkpoint_model_file_path: Path

In [28]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml,create_directories

In [29]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path= CONFIG_FILE_PATH,
                 params_file_path= PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def prepare_callbacks_config(self)-> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_chkpoint_dir = os.path.dirname(config.checkpoint_model_file_path)
        create_directories([Path(model_chkpoint_dir),
                            Path(config.tensorbaord_log_root_dir)
                            ])
        prepare_callbacks_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorbaord_log_root_dir= Path(config.tensorbaord_log_root_dir),
            checkpoint_model_file_path= Path(config.checkpoint_model_file_path)

        )
        return prepare_callbacks_config



In [30]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [31]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [32]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.prepare_callbacks_config
    prepare_callback = PrepareCallback(config=prepare_callback_config)
    callback_list = prepare_callback.get_tb_ckpt_callbacks
except Exception as e:
    raise e

[2024-03-24 14:57:23,082: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-24 14:57:23,085: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-24 14:57:23,085: INFO: common: created directory at: artifacts]
